In [15]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from pandas_datareader import data as pdr
yf.pdr_override() # <== that's all it takes :-)
import numpy as np
import statistics
from pprint import pprint
from math import isnan

sp500_history = pd.read_excel('./sp500_rebalance_announcements.xlsx')
sp500_history['implementation_date'] = pd.to_datetime(sp500_history['implementation_date']) 
sp500_history['effective_date'] = pd.to_datetime(sp500_history['effective_date']) 
sp500_history['announcement_date'] = pd.to_datetime(sp500_history['announcement_date']) 
sp500_history = sp500_history[['announcement_date', 'implementation_date', 'effective_date', 'ticker', 'name', 'type']]

start_date = datetime.strptime('2015-12-20', '%Y-%m-%d')
end_date = datetime.strptime('2020-12-31', '%Y-%m-%d')

# greater than the start date and smaller than the end date
mask = (sp500_history['announcement_date'] > start_date) & (sp500_history['announcement_date'] <= end_date)
sp500_history = sp500_history.loc[mask]

# Sort
sp500_history = sp500_history.sort_values(['announcement_date', 'type'], ascending=[True, True])

sp500_history = sp500_history.drop_duplicates()
sp500_history = sp500_history.reset_index(drop=True)

sp500_history

,announcement_date,implementation_date,effective_date,ticker,name,type
0,2015-12-28,2016-01-04,NaT,TW,Willis Towers Watson,ADDED
1,2015-12-28,2016-01-04,NaT,FOSL,Fossil Group,DELETED
2,2016-01-13,2016-01-15,NaT,EXR,Extra Space Storage,ADDED
3,2016-01-13,2016-01-15,NaT,CB,Chubb,DELETED
4,2016-01-22,2016-01-29,NaT,FRT,Federal Realty Trust,ADDED
...,...,...,...,...,...,...
151,2020-02-27,NaT,2020-03-03,XEC,Cimarex Energy,DELETED
152,2020-03-31,NaT,2020-04-03,OTIS,Otis Worldwide,ADDED
153,2020-03-31,NaT,2020-04-03,CARR,Carrier Global,ADDED
154,2020-03-31,NaT,2020-04-06,RTN,Raytheon,DELETED


In [19]:
pd.isna(sp500_history.loc[0]['effective_date'])

True

In [4]:
calendar = pd.read_excel('./Nasdaq_Trading_Calendar.xlsx', sheet_name=None)
rebal_dates = []
for year in calendar:
    sheet = calendar[year]
    mask = sheet['S&P Indexes Rebalance S&P 500, S&P 400, and S&P 600'] == 1
    year_rebal_dates = sheet.loc[mask]
    for date in year_rebal_dates['Date'].values:
        rebal_dates.append(date)
pprint(rebal_dates)

[numpy.datetime64('2020-03-20T00:00:00.000000000'),
 numpy.datetime64('2020-06-19T00:00:00.000000000'),
 numpy.datetime64('2020-09-18T00:00:00.000000000'),
 numpy.datetime64('2020-12-18T00:00:00.000000000'),
 numpy.datetime64('2019-03-15T00:00:00.000000000'),
 numpy.datetime64('2019-06-21T00:00:00.000000000'),
 numpy.datetime64('2019-09-20T00:00:00.000000000'),
 numpy.datetime64('2019-12-20T00:00:00.000000000'),
 numpy.datetime64('2018-03-16T00:00:00.000000000'),
 numpy.datetime64('2018-06-15T00:00:00.000000000'),
 numpy.datetime64('2018-09-21T00:00:00.000000000'),
 numpy.datetime64('2018-12-21T00:00:00.000000000'),
 numpy.datetime64('2017-03-17T00:00:00.000000000'),
 numpy.datetime64('2017-06-16T00:00:00.000000000'),
 numpy.datetime64('2017-09-15T00:00:00.000000000'),
 numpy.datetime64('2017-12-15T00:00:00.000000000'),
 numpy.datetime64('2016-03-18T00:00:00.000000000'),
 numpy.datetime64('2016-06-17T00:00:00.000000000'),
 numpy.datetime64('2016-09-16T00:00:00.000000000'),
 numpy.datet

In [5]:
# rebal_type is 'regular' or 'ad_hoc'
# add_delete is 'add' or 'delete'
# entry_date and exit_date are in terms of number of days before (-) or after (+) effective date
# entry_time and exit_time are 'Open' or 'Close'
strategy_attributes = ['rebal_type', 'add_delete', 'entry_date', 'entry_time', 'exit_date', 'exit_time']

impl_output_columns = ['implementation_date', 'effective_date']
impl_output_columns.extend(strategy_attributes)
impl_output_columns.extend(['total', 'up', 'count'])

impl_output = pd.DataFrame(columns=impl_output_columns)

ann_output_columns = ['announcement_date']
ann_output_columns.extend(strategy_attributes)
ann_output_columns.extend(['total', 'up', 'count'])

ann_output = pd.DataFrame(columns=ann_output_columns)

In [6]:
def update_impl_output_table(df_output, impl_date, eff_date, rebal_type, add_delete, 
                        entry_date, entry_time, exit_date, exit_time, price_change):
    if price_change > 0:
        up = 1
    else:
        up = 0

    conditions = ((df_output['implementation_date']==impl_date) & 
                 (df_output['rebal_type']==rebal_type) & 
                 (df_output['add_delete']==add_delete) &
                (df_output['entry_date']==entry_date) & 
                (df_output['entry_time']==entry_time) & 
                (df_output['exit_date']==exit_date) & 
                (df_output['exit_time']==exit_time))

    # new row
    if not conditions.any():
        data = [{'implementation_date': impl_date, 'rebal_type': rebal_type, 'add_delete': add_delete, 
                 'entry_date': entry_date, 'entry_time': entry_time, 
                 'exit_date': exit_date, 'exit_time': exit_time, 
                 'total': price_change, 'up': up, 'count': 1, 'effective_date': eff_date}]
        df_output = df_output.append(pd.DataFrame(data), ignore_index=True)

    # updating existing row
    else:
        df_output.loc[conditions, 'total'] += price_change
        df_output.loc[conditions, 'up'] += up
        df_output.loc[conditions, 'count'] += 1
    
    return df_output

In [7]:
def update_ann_output_table(df_output, announcement_date, rebal_type, add_delete, 
                        entry_date, entry_time, exit_date, exit_time, price_change):
    if price_change > 0:
        up = 1
    else:
        up = 0

    conditions = ((df_output['announcement_date']==announcement_date) & 
                 (df_output['rebal_type']==rebal_type) & 
                 (df_output['add_delete']==add_delete) &
                (df_output['entry_date']==entry_date) & 
                (df_output['entry_time']==entry_time) & 
                (df_output['exit_date']==exit_date) & 
                (df_output['exit_time']==exit_time))

    # new row
    if not conditions.any():
        data = [{'announcement_date': announcement_date, 'rebal_type': rebal_type, 'add_delete': add_delete, 
                 'entry_date': entry_date, 'entry_time': entry_time, 
                 'exit_date': exit_date, 'exit_time': exit_time, 
                 'total': price_change, 'up': up, 'count': 1}]
        df_output = df_output.append(pd.DataFrame(data), ignore_index=True)

    # updating existing row
    else:
        df_output.loc[conditions, 'total'] += price_change
        df_output.loc[conditions, 'up'] += up
        df_output.loc[conditions, 'count'] += 1
    
    return df_output

In [8]:
def find_price_at_index(prices, date_index, index_offset, time_of_day):
    '''
    Returns -1 if index not valid
    '''
    adj_index = date_index + index_offset
    if adj_index < 0 or adj_index >= len(prices):
        return -1
    price = prices.iloc[adj_index][time_of_day]
    if isnan(price):
        return -1
    return price

In [9]:
def find_price_change(prices, date_index, entry_date, entry_time, exit_date, exit_time):
    '''
    Returns percentage change in price between entry and exit
    Or NaN if anything invalid
    '''
    entry_price = find_price_at_index(prices, date_index, entry_date, entry_time)
    if entry_price < 0 or isnan(entry_price):
        return float('NaN')
    exit_price = find_price_at_index(prices, date_index, exit_date, exit_time)
    if exit_price < 0 or isnan(exit_price):
        return float('NaN')
    return exit_price / entry_price - 1 

In [20]:
days_bef_aft = 20 # the range of days before and after effective date to pull from Yahoo finance

exit_time = 'Close'

times_of_day = ['Open', 'Close']

has_impl_date = False

for idx, row in sp500_history.iterrows():
    # Has implementation date, but not effective date
    if pd.isna(row['effective_date']) and not pd.isna(row['implementation_date']):
        has_impl_date = True
        impl_date = row['implementation_date']

    # Has effective date but not implementation date
    elif pd.isna(row['implementation_date']) and not pd.isna(row['effective_date']):
        has_impl_date = False
        eff_date = row['effective_date']
        
    else:
        print("Error: Has neither implementation or effective date.")
        print(row)
        continue
    
    tickers = row['ticker'].split(';')
    for ticker in tickers:
        
        # End date is exclusive, so need to increase by 1
        if has_impl_date:
            impl_prices = pdr.get_data_yahoo(ticker, start=impl_date - timedelta(days=days_bef_aft), end=impl_date + timedelta(days=days_bef_aft + 1))
        else:
            impl_prices = pdr.get_data_yahoo(ticker, start=eff_date - timedelta(days=days_bef_aft), end=eff_date + timedelta(days=days_bef_aft + 1))
        
        if len(impl_prices) == 0:
            continue

        # Find the implementation date or effective date whichever is not available in the data
        if has_impl_date:
            try:
                eff_date_index = impl_prices.index.get_loc(impl_date + timedelta(days=1),method='backfill')
            except:
                print("Error: No date after implementation date was found among yahoo impl_prices")
                print(impl_date)
                continue
            if eff_date_index < 0:
                continue
            eff_date = impl_prices.iloc[eff_date_index].name

        else:
            try:
                impl_date_index = impl_prices.index.get_loc(eff_date - timedelta(days=1),method='pad')
            except:
                print("Error: No date before effective date was found among yahoo impl_prices")
                print(eff_date)
                continue
            if impl_date_index < 0:
        #         print('implementation date not in range')
        #         print(impl_prices)
        #         print(row['effective_date'])
                continue
            impl_date = impl_prices.iloc[impl_date_index].name
    #     print("sanity check:")
    #     print("impl date:")
    #     print(impl_date)
    #     print("eff date:")
    #     print(row['effective_date'])

        if impl_date in rebal_dates:
            rebal_type = 'regular'
    #         print("regular!")
    #         print(impl_prices)
    #         print(impl_date)
        else:
            rebal_type = 'ad_hoc'

        if row['type'] == 'ADDED':
            add_delete = 'add'
        else:
            add_delete = 'delete'

        '''
        Loop of different entry and exit dates for implementation date
        '''
        entry_date = max(-4, -impl_date_index)
        exit_date = 0
        while entry_date < 0:
            for time_of_day in times_of_day:
                entry_time = time_of_day

                price_change = find_price_change(impl_prices, impl_date_index, entry_date, entry_time, exit_date, exit_time)
                if isnan(price_change):
    #                 print("price change is invalid")
    #                 print(price_change)
                    continue

    #             if price_change == 0:
    #                 print('WHY EQUAL')
    #                 print(impl_prices)
    #                 print(entry_price)
    #                 print(exit_price)

                impl_output = update_impl_output_table(impl_output, impl_date, eff_date, rebal_type, add_delete, 
                            entry_date, entry_time, exit_date, exit_time, price_change)

            entry_date += 1

        entry_date = 0
        exit_date = min(4, len(impl_prices) - 1)
        while exit_date > 0:
            for time_of_day in times_of_day:
                entry_time = time_of_day

                price_change = find_price_change(impl_prices, impl_date_index, entry_date, entry_time, exit_date, exit_time)
                if isnan(price_change):
    #                 print("price change is invalid")
    #                 print(price_change)
                    continue

                impl_output = update_impl_output_table(impl_output, impl_date, eff_date, rebal_type, add_delete, 
                            entry_date, entry_time, exit_date, exit_time, price_change)

            exit_date -= 1

[*********************100%***********************]  1 of 1 completed

1 Failed download:
- TW: Data doesn't exist for startDate = 1450108800, endDate = 1453651200
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- BRCM: No data found for this date range, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- PCP: Data doesn't exist for startDate = 1452268800, endDate = 1455811200
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%*********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- GGP: No data found for this date range, symbol may be delisted
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- WCG: No data found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************

In [21]:
def get_uniques(input_list):
    # insert the list to the set 
    unique_set = set(input_list) 
    # convert the set to the list 
    unique_list = (list(unique_set)) 
    return unique_list

In [22]:
'''
Calculate population statistics
'''

stats_df_columns = []
stats_df_columns.extend(strategy_attributes)
stats_df_columns.extend(['total', 'up', 'count', 'mean', 'std', 'max', 'min', 'sharpe', 'uppct'])

stats_df = pd.DataFrame(columns=stats_df_columns)

# get the unique values for each strategy attribute
strategy_attribute_unique_values = map(lambda strategy_attribute: get_uniques(impl_output[strategy_attribute]), strategy_attributes)   

import itertools
# find strategies consisting of different combinations of unique values for each strategy attribute
strategies = list(itertools.product(*strategy_attribute_unique_values)) 

for strategy in strategies:
    # find the relevant rows for each strategy 
    relevant_rows = impl_output.loc[(impl_output[strategy_attributes]==strategy).all(axis=1), :]
    if len(relevant_rows) == 0:
        continue
    
    relevant_rows_sum = relevant_rows.loc[:, 'total':].sum(axis=0)
    mean = relevant_rows_sum['total'] / relevant_rows_sum['count']
    if relevant_rows_sum['count'] > 1:
        std = statistics.stdev(relevant_rows['total'])
    else:
        std = 0
    max_val = max(relevant_rows['total'])
    min_val = min(relevant_rows['total'])
    if std == 0:
        sharpe = mean / 0.00001 * np.sqrt(252)
    else:
        sharpe = mean / std * np.sqrt(252)
    uppct = relevant_rows_sum['up'] / relevant_rows_sum['count']
    strategy_dict = dict(zip(strategy_attributes, strategy)) 
    data = {'total': relevant_rows_sum['total'], 'up': relevant_rows_sum['up'], 'count': relevant_rows_sum['count'], 
            'mean': mean, 'std': std, 'max': max_val, 'min': min_val, 'sharpe': sharpe, 'uppct': uppct} 
    data.update(strategy_dict)
    stats_df = stats_df.append(pd.DataFrame([data]), ignore_index=True)


In [23]:
'''
Writing to excel
'''
writer = pd.ExcelWriter('sp500_impl_analysis.xlsx', engine='xlsxwriter')
impl_output.to_excel(writer, sheet_name='data')
stats_df.to_excel(writer, sheet_name='strategy_stats')
writer.save()